In [35]:
import numpy as np
import statsmodels.tsa.vector_ar.svar_model as svar
import pandas as pd

In [36]:
df = pd.read_csv("test_data.csv")

In [41]:
df.iloc[:, 1:4]

,SCU,ZCF,SAL
0,996.753421,1003.170356,998.269699
1,1001.293784,1007.682154,1014.257589
2,982.139811,1009.394196,1007.667311
3,996.690881,1006.246852,1009.591429
4,985.364314,1001.082492,1004.444957
5,980.535727,1000.985889,1011.221115
6,978.590794,999.121026,1010.768766
7,984.983083,1008.509892,1012.951237
8,980.623953,1002.127934,1024.336438
9,969.117332,1007.499176,1020.349449


In [60]:
sv = svar.SVAR(df.iloc[:, 1:4].values, svar_type='A', A=np.array(([1, 'e', 'e'], [0, 1, 'e'], [0, 0, 1])).T)

In [61]:
np.array(([1, 'e', 'e'], [0, 1, 'e'], [0, 0, 1]))

array([['1', 'e', 'e'],
       ['0', '1', 'e'],
       ['0', '0', '1']], dtype='|S11')

In [63]:
result = sv.fit()

In [64]:
dir(result)

['A',
 'A_mask',
 'A_solve',
 'B',
 'B_mask',
 'B_solve',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bmat_forc_cov',
 '_char_mat',
 '_chol_sigma_u',
 '_cov_alpha',
 '_cov_sigma',
 '_forecast_vars',
 '_model_type',
 '_omega_forc_cov',
 '_zz',
 'acf',
 'acorr',
 'aic',
 'bic',
 'bse',
 'coefs',
 'cov_params',
 'cov_ybar',
 'dates',
 'detomega',
 'df_model',
 'df_resid',
 'endog',
 'endog_lagged',
 'exog_names',
 'fevd',
 'fittedvalues',
 'forecast',
 'forecast_cov',
 'forecast_interval',
 'fpe',
 'get_eq_index',
 'hqic',
 'info_criteria',
 'intercept',
 'irf',
 'irf_errband_mc',
 'irf_resim',
 'is_stable',
 'k_ar',
 'k_trend',
 'llf',
 'long_run_effects',
 'ma_rep',
 'mean',
 'model',
 'mse',
 'n_totobs',
 'names',
 'neqs',
 'nobs',
 'orth_ma_rep',
 'params',
 '

In [73]:
rirf = result.irf()

In [78]:
result.long_run_effects()

array([[ 136.2326591 ,  279.19060015,   22.75189861],
       [ 228.77457736,  418.75104959, -435.22533762],
       [-226.26194906,  180.59371138,  715.83511762]])

In [6]:
a = np.array([1, 2, 1])
b = np.array([1, 2, 3])

In [12]:
t = np.logical_or(a == 1, b == 1)

In [16]:
tnum = np.zeros_like(t, dtype=float)
tnum[~t] = A[~A_mask]
Anum[A_mask] = np.nan

array([False,  True, False])

In [18]:
np.identity((10))

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])